In [1]:
import pandas as pd

from scripts.analysis import data

repos = data.get_fake_star_repos_all()
repos_dagster = pd.read_csv("data/dagster_test/fake_stars_complex_repos.csv")
stars_dagster = pd.read_csv("data/dagster_test/fake_stars_complex_users.csv")

repos_dagster = repos_dagster[(repos_dagster.n_fake_stars >= 50)]
repo_ids_dagster = set(repos_dagster.repo_id)

print(
    len(repo_ids_dagster),
    len(set(repos.repo_id) & repo_ids_dagster),
    repo_ids_dagster - set(repos.repo_id),
)
perfs = []
for repo_id in set(repos.repo_id) & repo_ids_dagster:
    repo_names = set(repos[repos.repo_id == repo_id].repo_name)
    fakes_copycatch, fakes_dagster = set(), set()
    for repo_name in repo_names:
        df = data.get_stars_from_repo(repo_name)
        if df is None:
            continue
        fakes_copycatch |= set(df[df.low_activity | df.clustered].actor)
        fakes_dagster |= set(
            stars_dagster[
                (stars_dagster.repo_name == repo_name)
                & (stars_dagster.fake_acct != "unknown")
            ].actor
        )
    perfs.append(
        {
            "repo_names": repo_names,
            "n_fakes_copycatch": len(fakes_copycatch),
            "n_fakes_dagster": len(fakes_dagster),
            "n_overlap": len(fakes_copycatch & fakes_dagster),
        }
    )
perfs = pd.DataFrame(perfs)
print("Precision:", perfs.n_overlap.sum() / perfs.n_fakes_copycatch.sum())
print("Recall:", perfs.n_overlap.sum() / perfs.n_fakes_dagster.sum())
perfs

44 41 {'MDEwOlJlcG9zaXRvcnkyMTM2NDkyMTM=', 'R_kgDOHxZ6lw', 'MDEwOlJlcG9zaXRvcnk4OTkxMzY1NQ=='}
Precision: 0.7433702689243028
Recall: 0.21571018262613126


,repo_names,n_fakes_copycatch,n_fakes_dagster,n_overlap
0,{ccxt/ccxt},350,646,350
1,{nocodb/nocodb},154,551,154
2,{h5bp/html5-boilerplate},238,1393,238
3,{mrdoob/three.js},1179,2685,494
4,{tabler/tabler},118,411,118
5,{Joystream/joystream},676,1115,676
6,{tailwindlabs/tailwindcss},440,2896,173
7,{electron/electron},419,3389,419
8,{vercel/next.js},1274,5753,558
9,{hydro-dev/Hydro},609,674,609
